# Formbook Process Creation
Detects Formbook like process executions that inject code into a set of files in the System32 folder, which executes a special command command line to delete the dropper from the AppData Temp folder. We avoid false positives by excluding all parent process with command line parameters.

## Rule Content
```
- title: Formbook Process Creation
  id: 032f5fb3-d959-41a5-9263-4173c802dc2b
  status: experimental
  description: Detects Formbook like process executions that inject code into a set
    of files in the System32 folder, which executes a special command command line
    to delete the dropper from the AppData Temp folder. We avoid false positives by
    excluding all parent process with command line parameters.
  author: Florian Roth
  date: 2019/09/30
  modified: 2019/10/31
  references:
  - https://inquest.net/blog/2018/06/22/a-look-at-formbook-stealer
  - https://app.any.run/tasks/388d5802-aa48-4826-b069-250420504758/
  - https://app.any.run/tasks/8e22486b-5edc-4cef-821c-373e945f296c/
  - https://app.any.run/tasks/62bb01ae-25a4-4180-b278-8e464a90b8d7/
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentCommandLine:
      - C:\Windows\System32\\*.exe
      - C:\Windows\SysWOW64\\*.exe
      CommandLine:
      - '* /c del "C:\Users\\*\AppData\Local\Temp\\*.exe'
      - '* /c del "C:\Users\\*\Desktop\\*.exe'
      - '* /C type nul > "C:\Users\\*\Desktop\\*.exe'
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Unknown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_parent_command_line.keyword:(C\:\\Windows\\System32\\*.exe OR C\:\\Windows\\SysWOW64\\*.exe) AND process_command_line.keyword:(*\ \/c\ del\ \"C\:\\Users\\*\\AppData\\Local\\Temp\\*.exe OR *\ \/c\ del\ \"C\:\\Users\\*\\Desktop\\*.exe OR *\ \/C\ type\ nul\ \ \"C\:\\Users\\*\\Desktop\\*.exe))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()